In [2]:
%load_ext dotenv
%dotenv
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import boto3


s3_client = boto3.client(
    's3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
)
BUCKET_NAME = 'tnn-hse-medtech'
DATASET_DIR = 'datasets/'


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [3]:
data = pd.read_csv('normalized_data.csv')

In [25]:
from skimage.feature import hog
from PIL import Image

def hog_picture(image: Image):
    array = np.asarray(image)
    vector = hog(
        array,
        orientations=8,
        pixels_per_cell=(16, 16),
        cells_per_block=(1, 1),
        feature_vector=True,
    )
    return vector


In [26]:
import io
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
from PIL import Image, ImageOps
from ipywidgets import IntProgress
from IPython.display import display

@dataclass()
class Metadata:
    severity: int
    relative_path: str
    hog: np.array

def handle_file(item) -> Metadata:
    buffer = io.BytesIO()
    s3_client.download_fileobj(BUCKET_NAME, f'{DATASET_DIR}{item.relative_path}', buffer)
    buffer.seek(0)
    img = Image.open(buffer).convert('L')
    hog_fd = hog_picture(img)
    return Metadata(item.severity, item.relative_path, hog_fd)

raw_data = []
progress = IntProgress(min=0, max=len(data))
display(progress)
with ThreadPoolExecutor(max_workers=50) as pool:
    for item in pool.map(handle_file, data.itertuples()):
        raw_data.append(item)
        progress.value += 1


IntProgress(value=0, max=3278)

In [32]:
X_data = pd.DataFrame(data=(item.hog for item in raw_data))
y_target = pd.DataFrame(data=(item.severity for item in raw_data))

0    1006
1     954
2     464
3     442
4     412
Name: count, dtype: int64